# Importing Training Data and Preprocessing


In [1]:
from collections import Counter
import nltk
import pandas as pd
import numpy as np
import collections, numpy
import mlxtend
import matplotlib
from matplotlib import pyplot
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
matplotlib.rcParams['figure.figsize'] = (10, 10)


from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from nltk.tokenize import sent_tokenize
import pandas as pd
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
import re
import xml.etree.ElementTree as ET


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = "/content/drive/MyDrive/Georgetown/Massv_Health_Data_Fund/Final_Project/smoking_status_train.xml"
test_path = "/content/drive/MyDrive/Georgetown/Massv_Health_Data_Fund/Final_Project/smoking_status_test.xml"
test_path_labeled = "/content/drive/MyDrive/Georgetown/Massv_Health_Data_Fund/Final_Project/smoking_status_test_with_labels.xml"

def read_xml_custom(file_path, df_name):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Extract records into a list of dictionaries
    records = []
    for record in root.findall('RECORD'):
        record_id = record.get('ID')
        smoking_status = record.find('SMOKING').get('STATUS') if record.find('SMOKING') is not None else None
        text_content = record.find('TEXT').text.replace('\n', ' ').strip() if record.find('TEXT') is not None else None

        records.append({'ID': record_id, 'SMOKING': smoking_status, 'TEXT': text_content})

    # Create and return a DataFrame
    return pd.DataFrame(records)

# Example usage
data_set = read_xml_custom(data_path, 'data_set')

data_set.head()

,ID,SMOKING,TEXT
0,641,CURRENT SMOKER,977146916 HLGMC 2878891 022690 01/27/1997 12:0...
1,643,CURRENT SMOKER,026738007 CMC 15319689 3/25/1998 12:00:00 AM M...
2,681,CURRENT SMOKER,071962960 BH 4236518 417454 12/10/2001 12:00:0...
3,704,CURRENT SMOKER,418520250 NVH 61562872 3/11/1995 12:00:00 AM D...
4,757,CURRENT SMOKER,301443520 CTMC 49020928 448922 1/11/1990 12:00...


In [5]:
from sklearn.model_selection import train_test_split

# Split the DataFrame into train and validation sets (80%-20%)
train_set, val_set = train_test_split(data_set, test_size=0.2, random_state=42, stratify=data_set['SMOKING'])

# Verify the shapes of the splits
print("Train Set Shape:", train_set.shape)
print("Validation Set Shape:", val_set.shape)


Train Set Shape: (318, 3)
Validation Set Shape: (80, 3)


In [6]:
train_set.head()

,ID,SMOKING,TEXT
199,921,UNKNOWN,779878634 TGCHO 9260099 612765 12/11/2003 12:0...
168,841,UNKNOWN,747495092 CTMC 91057356 183023 7/3/1998 12:00:...
56,620,NON-SMOKER,791411240 PRGH 80504780 383201 9/16/2000 12:00...
166,837,UNKNOWN,223159990 PUOMC 3546292 181083 366324 7/31/200...
130,749,UNKNOWN,736743101 PUOMC 2427050 19324/7383 418074 4/17...


In [7]:
val_set.head()

,ID,SMOKING,TEXT
362,303,UNKNOWN,745943853 SC 59881622 2458882 2/28/2005 12:00:...
117,719,UNKNOWN,602064980 PUOMC 5102918 20144/f643 339700 12/1...
89,596,SMOKER,348983165 FIH 4912803 97509/32z1 028205 4/14/1...
345,205,UNKNOWN,334094996 YC 33195888 9046143 03/25/2003 12:00...
134,758,UNKNOWN,649008161 CTMC 07519742 1990961 8/24/2004 12:0...


# Importing and Preparing Test Set


In [8]:
test_set_labeled = read_xml_custom(test_path_labeled, 'test_set')
test_set_labeled.head()

,ID,SMOKING,TEXT
0,660,PAST SMOKER,156406283 HLGMC 7213645 64723/51cy 5/28/1993 1...
1,845,CURRENT SMOKER,487197293 PUOMC 2466262 408602 1976045 4/26/20...
2,515,CURRENT SMOKER,176318078 FIH 4189699 28872/d70h 094756 11/03/...
3,562,CURRENT SMOKER,245096078 FIH 9174858 12/February 997359 11/3/...
4,633,CURRENT SMOKER,237073320 FIH 9746390 797120 251605 01/26/1998...


### Feature Engineering Cont'd. BERT

At this point we have:
- train_set as the dataframe with cleaned tokens from sentence tokens
- train_tfidf as the tfidf vector based on the train set
- test_set_labeled as the df with cleaned tokens from smoking sentence tokens and
- test_tfidf as the tfidf vector for test set

In [9]:
train_set.head()

,ID,SMOKING,TEXT
199,921,UNKNOWN,779878634 TGCHO 9260099 612765 12/11/2003 12:0...
168,841,UNKNOWN,747495092 CTMC 91057356 183023 7/3/1998 12:00:...
56,620,NON-SMOKER,791411240 PRGH 80504780 383201 9/16/2000 12:00...
166,837,UNKNOWN,223159990 PUOMC 3546292 181083 366324 7/31/200...
130,749,UNKNOWN,736743101 PUOMC 2427050 19324/7383 418074 4/17...


In [10]:
test_set_labeled.head()

,ID,SMOKING,TEXT
0,660,PAST SMOKER,156406283 HLGMC 7213645 64723/51cy 5/28/1993 1...
1,845,CURRENT SMOKER,487197293 PUOMC 2466262 408602 1976045 4/26/20...
2,515,CURRENT SMOKER,176318078 FIH 4189699 28872/d70h 094756 11/03/...
3,562,CURRENT SMOKER,245096078 FIH 9174858 12/February 997359 11/3/...
4,633,CURRENT SMOKER,237073320 FIH 9746390 797120 251605 01/26/1998...


In [63]:
# importing libraries
import random
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn as nn
from sklearn.utils.class_weight import compute_class_weight

In [12]:
# Set a random seed
random_seed = 42
random.seed(random_seed)

# Set a random seed for PyTorch (for GPU as well)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

In [13]:
# Load BERT tokenizer and model - tried using emilyalsentzer/Bio_ClinicalBERT
tokenizer = BertTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
model = BertModel.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [14]:
#encoder

# tokenize and encode sequences in the training set
bert_tokens_train = tokenizer.batch_encode_plus(
    train_set["TEXT"],
    max_length = 25,
    padding = 'max_length',
    return_tensors='pt',
    truncation=True
)
# tokenize and encode sequences in the test set
bert_tokens_test = tokenizer.batch_encode_plus(
    test_set_labeled["TEXT"],
    max_length = 25,
    padding = 'max_length',
    return_tensors='pt',
    truncation=True
)

bert_tokens_val = tokenizer.batch_encode_plus(
    val_set["TEXT"],
    max_length = 25,
    padding = 'max_length',
    return_tensors='pt',
    truncation=True
)



In [15]:
# before we convert the lists above to tensors, we must first convert our labels into numeric format.
# this is because tensors from pytorch don't know how to handle numeric data

import torch
from sklearn.preprocessing import LabelEncoder

# Initialize a label encoder
label_encoder = LabelEncoder()

# Fit the encoder on the training data and transform it
train_set['SMOKING_encoded'] = label_encoder.fit_transform(train_set['SMOKING'])

val_set['SMOKING_encoded'] = label_encoder.transform(val_set['SMOKING'])

# Apply the same encoding to the test data (using transform, not fit_transform)
test_set_labeled['SMOKING_encoded'] = label_encoder.transform(test_set_labeled['SMOKING'])


In [21]:
print(" ".join(map(str, train_set['SMOKING_encoded'].tolist())))
print(" ".join(map(str, train_set['SMOKING'].tolist())))

4 4 1 4 4 3 4 1 4 4 1 4 4 4 4 4 4 2 2 4 4 4 0 4 4 4 1 4 4 4 0 4 4 1 4 4 4 4 2 1 4 4 4 4 1 4 4 4 0 0 1 1 4 0 4 4 4 4 4 0 1 4 4 4 4 2 4 1 2 4 2 0 4 4 1 4 1 4 1 0 4 4 2 1 0 4 4 4 4 4 2 4 4 2 4 4 1 4 1 1 4 4 1 4 1 4 4 4 4 4 1 4 4 4 1 0 1 4 4 1 4 4 0 2 4 4 4 4 4 4 4 4 4 1 2 4 1 4 4 0 4 4 4 4 4 3 2 1 4 0 0 4 4 4 4 0 4 4 3 4 1 4 0 1 4 4 4 0 3 4 1 4 4 4 1 2 4 4 4 4 0 4 4 4 1 3 4 4 2 3 4 4 4 2 0 2 1 1 4 1 2 4 4 1 1 1 1 4 4 1 1 0 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 0 1 0 0 0 4 4 4 4 4 4 4 4 1 4 4 0 0 4 4 1 2 1 2 4 4 4 4 4 4 2 4 4 4 4 2 4 4 1 4 1 4 4 4 4 2 1 4 4 2 4 0 4 4 4 4 4 3 4 4 2 4 4 4 4 4 2 4 4 4 1 4 4 2 0 2 1 1 4 2 1 4 4 4 4 1 4 4 4 4
UNKNOWN UNKNOWN NON-SMOKER UNKNOWN UNKNOWN SMOKER UNKNOWN NON-SMOKER UNKNOWN UNKNOWN NON-SMOKER UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN PAST SMOKER PAST SMOKER UNKNOWN UNKNOWN UNKNOWN CURRENT SMOKER UNKNOWN UNKNOWN UNKNOWN NON-SMOKER UNKNOWN UNKNOWN UNKNOWN CURRENT SMOKER UNKNOWN UNKNOWN NON-SMOKER UNKNOWN UNKNOWN UNKNOWN UNKNOWN PAST SMOKER NON-SMOKER U

In [22]:
print(" ".join(map(str, val_set['SMOKING_encoded'].tolist())))
print(" ".join(map(str, val_set['SMOKING'].tolist())))

4 4 3 4 4 4 4 4 4 4 2 2 4 1 4 4 4 0 4 0 4 4 4 4 4 1 1 4 2 4 4 4 4 4 1 4 4 4 1 4 3 4 4 4 4 0 1 4 4 4 1 1 4 1 1 4 1 4 4 4 2 4 0 4 0 0 4 2 0 4 4 2 4 4 2 1 4 1 4 4
UNKNOWN UNKNOWN SMOKER UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN PAST SMOKER PAST SMOKER UNKNOWN NON-SMOKER UNKNOWN UNKNOWN UNKNOWN CURRENT SMOKER UNKNOWN CURRENT SMOKER UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN NON-SMOKER NON-SMOKER UNKNOWN PAST SMOKER UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN NON-SMOKER UNKNOWN UNKNOWN UNKNOWN NON-SMOKER UNKNOWN SMOKER UNKNOWN UNKNOWN UNKNOWN UNKNOWN CURRENT SMOKER NON-SMOKER UNKNOWN UNKNOWN UNKNOWN NON-SMOKER NON-SMOKER UNKNOWN NON-SMOKER NON-SMOKER UNKNOWN NON-SMOKER UNKNOWN UNKNOWN UNKNOWN PAST SMOKER UNKNOWN CURRENT SMOKER UNKNOWN CURRENT SMOKER CURRENT SMOKER UNKNOWN PAST SMOKER CURRENT SMOKER UNKNOWN UNKNOWN PAST SMOKER UNKNOWN UNKNOWN PAST SMOKER NON-SMOKER UNKNOWN NON-SMOKER UNKNOWN UNKNOWN


In [23]:
print(" ".join(map(str, test_set_labeled['SMOKING_encoded'].tolist())))
print(" ".join(map(str, test_set_labeled['SMOKING'].tolist())))

2 0 0 0 0 1 1 1 1 1 1 1 1 0 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 1 1 1 2 2 2 2 3 3 4 4 4 4 4 4 4 0 3 0 0 0 0 1 1 1 1 1 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
PAST SMOKER CURRENT SMOKER CURRENT SMOKER CURRENT SMOKER CURRENT SMOKER NON-SMOKER NON-SMOKER NON-SMOKER NON-SMOKER NON-SMOKER NON-SMOKER NON-SMOKER NON-SMOKER CURRENT SMOKER PAST SMOKER PAST SMOKER PAST SMOKER UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN CURRENT SMOKER NON-SMOKER NON-SMOKER NON-SMOKER PAST SMOKER PAST SMOKER PAST SMOKER PAST SMOKER SMOKER SMOKER UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN UNKNOWN CURRENT SMOKER SMOKER CURRENT SMOKER CURRENT SMOKER CURRENT SMOKER CURRENT SMOKER NON-SMOKER NON-SMOKER NON-SMOKER NON-SMOKER N

In [25]:
# based on the above, the encoding scheme has the following key values

# 0 = Current Smoker
# 1 = Non Smoker
# 2 = Past Smoker
# 3 = Smoker
# 4 = Unknown

In [27]:
## convert lists to tensors

train_seq = torch.tensor(bert_tokens_train['input_ids'])
train_mask = torch.tensor(bert_tokens_train['attention_mask'])
train_y = torch.tensor(train_set['SMOKING_encoded'].tolist())

val_seq = torch.tensor(bert_tokens_val['input_ids'])
val_mask = torch.tensor(bert_tokens_val['attention_mask'])
val_y = torch.tensor(val_set['SMOKING_encoded'].tolist())

test_seq = torch.tensor(bert_tokens_test['input_ids'])
test_mask = torch.tensor(bert_tokens_test['attention_mask'])
test_y = torch.tensor(test_set_labeled['SMOKING_encoded'].tolist())

<ipython-input-27-854a847b18d8>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_seq = torch.tensor(bert_tokens_train['input_ids'])
<ipython-input-27-854a847b18d8>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_mask = torch.tensor(bert_tokens_train['attention_mask'])
<ipython-input-27-854a847b18d8>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_seq = torch.tensor(bert_tokens_val['input_ids'])
<ipython-input-27-854a847b18d8>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().d

In [28]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [29]:
# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [30]:
for param in model.parameters():
    param.requires_grad = False

In [31]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

# specify GPU
device = torch.device("cuda")

In [56]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
        super(BERT_Arch, self).__init__()

        self.bert = bert

        # dropout layer
        self.dropout = nn.Dropout(0.1)

        # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768,512)

        # dense layer 2 (Output layer) - Changed to 5 outputs
        self.fc2 = nn.Linear(512,5) # Updated to 5 output classes

        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

        #pass the inputs to the model
        _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False) # Use pooled_output

        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

        # output layer
        x = self.fc2(x)

        # apply softmax activation
        x = self.softmax(x)

        return x

In [33]:
# pass the pre-trained BERT to our define architecture
model_device = BERT_Arch(model)

# push the model to GPU
model_device = model_device.to(device)

In [34]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(),lr = 1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [35]:
!pip install --upgrade scikit-learn

import sklearn
print(sklearn.__version__)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 87.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.2
    Uninstalling scikit-learn-1.5.2:
      Successfully uninstalled scikit-learn-1.5.2
1.5.2


In [36]:
from sklearn.utils.class_weight import compute_class_weight

train_labels = train_set['SMOKING_encoded'].values

#compute the class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=np.array(train_labels))

print("Class Weights:",class_weights)

Class Weights: [2.27142857 1.2        2.19310345 9.08571429 0.31641791]


The classes are encoded as follows:

* 0 = Current Smoker | weight 2.27
* 1 = Non Smoker | weight 1.206
* 2 = Past Smoker | weight 2.21
* 3 = Smoker | weight 8.84
* 4 = Unknown | weight 0.316




In [37]:
# converting list of class weights to a tensor
weights= torch.tensor(class_weights,dtype=torch.float)

# push to GPU
weights = weights.to(device)

# define the loss function
cross_entropy  = nn.NLLLoss(weight=weights)

# number of training epochs
epochs = 10

### BERT Model Usage and Fine Tuning

In [64]:
# function to train the model
def train():
    model.train()
    total_loss = 0

    # empty list to save model predictions
    total_preds = []

    # iterate over batches
    for step, batch in enumerate(train_dataloader):
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # push the batch to gpu
        batch = [r.to(device) for r in batch]
        sent_id, mask, labels = batch

        model_device.zero_grad()

        # get model predictions for the current batch
        outputs = model_device(sent_id, mask)

        preds = model(sent_id, mask, token_type_ids)

        # Correctly extract logits (no need for slicing)
        logits = outputs  # CLS token embeddings for classification

        unique_labels_in_batch = torch.unique(labels)

        # Create a new weight tensor with only the weights for the classes present in the batch
        batch_weights = torch.tensor([weights[i.item()] for i in unique_labels_in_batch], device=device)

        # Create a temporary loss function with the batch-specific weights
        batch_cross_entropy = nn.NLLLoss(weight=batch_weights if len(batch_weights) > 1 else None, reduction='mean') # Use batch_weights or None, and mean reduction

        # Compute the loss
        loss = batch_cross_entropy(logits, labels)

        total_loss += loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # detach predictions and move to CPU
        preds = logits.detach().cpu().numpy()
        total_preds.append(preds)

    # compute the average loss
    avg_loss = total_loss / len(train_dataloader)

    # reshape predictions
    total_preds = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds


In [65]:
# function for evaluating the model
def evaluate():
    print("\nEvaluating...")
    model.eval()

    total_loss = 0
    total_preds = []

    for step, batch in enumerate(val_dataloader):
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

        batch = [t.to(device) for t in batch]
        sent_id, mask, labels = batch

        with torch.no_grad():
            outputs = model_device(input_ids=sent_id, attention_mask=mask)

            # extract logits
            logits = outputs

            unique_labels_in_batch = torch.unique(labels)

            # Create a new weight tensor with only the weights for the classes present in the batch
            batch_weights = torch.tensor([weights[i.item()] for i in unique_labels_in_batch], device=device)


            # Create a temporary loss function with the batch-specific weights
            batch_cross_entropy = nn.NLLLoss(weight=batch_weights if len(batch_weights) > 1 else None, reduction='mean') # Use batch_weights or None, and mean reduction


            # compute the loss
            loss = cross_entropy(logits, labels)
            total_loss += loss.item()

            # detach predictions and move to CPU
            preds = logits.detach().cpu().numpy()
            total_preds.append(preds)

    avg_loss = total_loss / len(val_dataloader)
    total_preds = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):

    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    #train model
    train_loss, _ = train()

    #evaluate model
    valid_loss, _ = evaluate()

    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')

    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


The BERT implementation of this project was not completed. For implementation of a LLM like BERT, refer to my project on drug repurposing assistant (Rx-ReWise).